In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install sentence_transformers

In [ ]:
import pandas as pd
import numpy as np
import pickle

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
%cd /content/drive/MyDrive/multicampus/team_project1/movies

/content/drive/MyDrive/multicampus/team_project1/movies


In [ ]:
!ls

 1205bert연습.ipynb     df.csv		        movies_embedded.csv
 app.py		        emo_dict.pkl	        movies_genres.csv
 chatbot_data2.csv      main.ui		        movies.ipynb
 chatbot_data.csv       movie_data.ipynb       '챗봇_train (1).ipynb'
 chatbot_embedded.csv   movie_recommand.ipynb   Untitled0.ipynb


In [ ]:
with open('emo_dict.pkl','rb') as f:
  emo_dict = pickle.load(f)

In [ ]:
chatbot_embedded = pd.read_csv('chatbot_data2.csv')
movies_embedded = pd.read_csv('movies_embedded.csv')

In [ ]:
model_st = SentenceTransformer('jhgan/ko-sroberta-multitask')

In [ ]:
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [ ]:
chatbot_embedded.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
chatbot_embedded['embedding'] = chatbot_embedded.apply(lambda x: model_st.encode(x['사람']), axis=1)

In [ ]:
movies_embedded = pd.read_csv('movies_embedded.csv')
movies_embedded['embedding'] = movies_embedded.apply(lambda x: model_st.encode(x['제목']), axis=1)

In [ ]:
import random
import torch
from torch.utils.data import DataLoader, Dataset

In [ ]:
%cd /content/drive/MyDrive/multicampus/team_project1/

/content/drive/MyDrive/multicampus/team_project1


In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device('cpu')

In [ ]:
device

device(type='cpu')

In [ ]:
data = pd.read_csv('data/train2.csv')
data.drop('sentiment2', axis=1, inplace=True)
data = data[:50]
data.head()

,sentiment1,chat,bot
0,불안,나 오늘 너무 황당했어.,어떤 일이 있으셨나요?
1,당황,오늘 아침에 딸이랑 싸웠어.,무슨 일로 싸우셨나요?
2,슬픔,심지어 이번에 중요한 프로젝트를 성공시켰는데도 기쁨이 전혀 없더라.,공들인 일이 결실을 맺었는데도 기쁘지가 않았다면 이제 어떤 것을 해보면 나에게 좋을까요?
3,분노,불평한 건 사과하고 요즘 부정적인 감정이 든단 사실을 솔직하게 말할래.,솔직한 마음을 표현하여 잘 해결되길 바랄게요.
4,불안,내가 개입하는 건 더 상황을 악화시킬 것 같아서 선생님께 한 번 말씀드려보려고.,선생님께 말씀 드려볼 생각이군요.


In [ ]:
data = data[['chat', 'bot', 'sentiment1']]
data.columns = ['Q', 'A', 'label']
data.head()

,Q,A,label
0,나 오늘 너무 황당했어.,어떤 일이 있으셨나요?,불안
1,오늘 아침에 딸이랑 싸웠어.,무슨 일로 싸우셨나요?,당황
2,심지어 이번에 중요한 프로젝트를 성공시켰는데도 기쁨이 전혀 없더라.,공들인 일이 결실을 맺었는데도 기쁘지가 않았다면 이제 어떤 것을 해보면 나에게 좋을까요?,슬픔
3,불평한 건 사과하고 요즘 부정적인 감정이 든단 사실을 솔직하게 말할래.,솔직한 마음을 표현하여 잘 해결되길 바랄게요.,분노
4,내가 개입하는 건 더 상황을 악화시킬 것 같아서 선생님께 한 번 말씀드려보려고.,선생님께 말씀 드려볼 생각이군요.,불안


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['label'] = le.fit_transform(data['label'])

In [ ]:
le.classes_

array(['기쁨', '당황', '분노', '불안', '상처', '슬픔'], dtype=object)

In [ ]:
emo_dict

{0: '불안할', 1: '화날', 2: '슬플', 3: '기쁠', 4: '상처 받았을', 5: '당황했을'}

In [ ]:
le.inverse_transform([3, 2, 5, 0, 4, 1])

array(['불안', '분노', '슬픔', '기쁨', '상처', '당황'], dtype=object)

In [ ]:
# Dataset 정의
# 챗봇 데이터를 처리하는 클래스
class ChatbotDataset(Dataset):
    def __init__(self, chats, max_len=40):  # 데이터셋 전처리
        self._data = chats
        self.max_len = max_len
        self.q_token = Q_TKN
        self.a_token = A_TKN
        self.sent_token = SENT
        self.eos = EOS
        self.mask = MASK
        self.tokenizer = koGPT2_TOKENIZER

    def __len__(self):  # chatbotdata 길이 리턴
        return len(self._data)

    def __getitem__(self, idx):  # 로드한 챗봇 데이터를 차례로 DataLoader로 넘겨주는 메서드
        turn = self._data.iloc[idx]
        q = turn["Q"]  # 질문을 가져온다.
        q = re.sub(r"([?.!,])", r" ", q)  # 구둣점 제거

        a = turn["A"]  # 답변을 가져온다.
        a = re.sub(r"([?.!,])", r" ", a)  # 구둣점 제거

        q_toked = self.tokenizer.tokenize(self.q_token + q + self.sent_token)
        q_len = len(q_toked)

        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
        a_len = len(a_toked)

        if q_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        labels = [self.mask,] * q_len + a_toked[1:]

        mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]

        token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]

        return (token_ids, np.array(mask), labels_ids)

In [ ]:
def collate_batch(batch):
    data = [item[0] for item in batch]
    mask = [item[1] for item in batch]
    label = [item[2] for item in batch]
    return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)

In [ ]:
### KoGPT2 챗봇 ###

In [ ]:
!pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 798 kB 5.0 MB/s 
     |████████████████████████████████| 512 kB 75.8 MB/s 
     |████████████████████████████████| 125 kB 69.3 MB/s 
     |████████████████████████████████| 87 kB 7.6 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115943 sha256=53d2648e01bac0a6bb1076191b6e741a7433ecc787355e9e12c502f74e6fb436
  Stored in directory: /root/.cache/pip/wheels/1f/10/06/2a990ee4d73a8479fe2922445e8a876d38cfbfed052284c6a1
Successfully built fire


In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.core.lightning import LightningModule
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
import re

In [ ]:
Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

In [ ]:
koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            bos_token=BOS, eos_token=EOS, unk_token='<unk>',
            pad_token=PAD, mask_token=MASK) 
model_gpt = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


Downloading:   0%|          | 0.00/513M [00:00<?, ?B/s]

In [ ]:
train_set = ChatbotDataset(data, max_len=40)
train_dataloader = DataLoader(train_set, batch_size=32, num_workers=1, shuffle=True, collate_fn=collate_batch, pin_memory=True)

In [ ]:
model_gpt.to(device)
model_gpt.train()

In [ ]:
learning_rate = 3e-5
criterion = torch.nn.CrossEntropyLoss(reduction="none").to(device)
optimizer = torch.optim.Adam(model_gpt.parameters(), lr=learning_rate)

epoch = 10
Sneg = -1e18

In [ ]:
print ("start")
for epoch in range(epoch):
    for batch_idx, samples in enumerate(train_dataloader):
        optimizer.zero_grad()
        token_ids, mask, label = samples
        out = model_gpt(token_ids)
        out = out.logits      # Returns a new tensor with the logit of the elements of input
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
        mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))
        loss = criterion(mask_out.transpose(2, 1), label)
        # 평균 loss 만들기 avg_loss[0] / avg_loss[1] <- loss 정규화
        avg_loss = loss.sum() / mask.sum()
        avg_loss.backward()
        # 학습 끝
        optimizer.step()
print ("end")

start


<ipython-input-24-495d11317178>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)
<ipython-input-24-495d11317178>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)
<ipython-input-24-495d11317178>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at 

end


In [ ]:
with torch.no_grad():
  while(1):
    q = input("user> ").strip()
    embedded = model_st.encode(q)

    chatbot_embedded['score'] = chatbot_embedded.apply(lambda x: cos_sim(x['embedding'], embedded), axis=1)
    genre = chatbot_embedded.iloc[chatbot_embedded['score'].idxmax(), 1]
    emotion = emo_dict[genre]

    sent = str(genre)
    # if q == "영화나 추천해봐.":
    if re.findall('(?=.*영화)(?=.*추천)', q):
      break
    a = ""
    while(1):
      input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode(Q_TKN + q + SENT + sent + A_TKN + a)).unsqueeze(dim=0)
      pred = model_gpt(input_ids)
      pred = pred.logits
      gen = koGPT2_TOKENIZER.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().numpy().tolist())[-1]
      if gen == EOS:
        break
      a += gen.replace("▁", " ")
    print("bot> {}".format(a.strip()))

  print(f'bot> {emotion} 때 보기 좋은 영화 추천해 드릴게요.')

  movies_embedded1 = movies_embedded.copy()
  movies_embedded1 = movies_embedded1[movies_embedded1['영화'] == genre]
  movies_embedded1['score'] = movies_embedded1.apply(lambda x: cos_sim(x['embedding'], embedded), axis=1)

  top3 = movies_embedded1.sort_values('score', ascending=False).iloc[:3, 0]

  for i, t in enumerate(top3.values):
    print(f'{i+1}. {t}')

user> 오늘 너무 힘들었어.
bot> 많이 힘드셨군요
user> 응응. 시험을 망쳤거든.
bot> 시험에선 어떻게 된 일인지 자세히 말씀해주시겠어요
user> 세상에 찍은 게 모조리 나가버렸네.
bot> 그거 좋은 생각이네요
user> 뭐야
bot> 어떤 점이요
user> 영화나 추천해봐.
bot> 슬플 때 보기 좋은 영화 추천해 드릴게요.
1. 국도극장: 감독판
2. 시, 나리오
3. 판타스틱 썸머
